In [3]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def process_game(game):
    """
    Processes a single game's data into a list of dictionaries.

    :param game: A game object containing team data.
    :return: List of dictionaries with team statistics for the game.
    """
    game_id = game.id
    teams_data = []

    for team in game.teams:
        stats_dict = {stat.category: stat.stat for stat in team.stats}
        teams_data.append({
            'game_id': game_id,
            'team_id': team.school_id,
            'team_name': team.school,
            'home_away': team.home_away,
            'points': team.points,
            **stats_dict
        })

    return teams_data

def extract_team_data(year, week):
    """
    Extracts team statistics and converts them directly into a DataFrame.

    :param year: Year of the game data.
    :param week: Week of the game data.
    :return: DataFrame containing extracted team statistics for each game.
    """
    # Retrieve game data from the API
    game_data = api_instance.get_team_game_stats(year=year, week=week)

    # Use parallel processing to handle the games
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_game, game_data))
    
    # Flatten the list of lists
    flattened_results = [item for sublist in results for item in sublist]
    
    # Create DataFrame directly from the processed data
    df = pd.DataFrame(flattened_results).fillna(0)  # Replace NaNs with 0 if needed

    return df

In [6]:
%run import_modules.ipynb
df=extract_team_data(year=2023, week=1)